# Causal Inference

In this notebook we will work with an example of the the pipeline of Causal Inference. We will proceed with the following steps:
1. Create a causal model
2. Simulate data from a causal model
3. Run a causal effect identification



### Installing R packages (colab)

In [6]:
cat(system('python3 -c "from google.colab import drive\ndrive.mount()"', intern=TRUE), sep='\n', wait=TRUE)

Warning message in system("python3 -c \"from google.colab import drive\ndrive.mount()\"", :
“running command 'python3 -c "from google.colab import drive
drive.mount()"' had status 1”



TRUE


In [5]:
system("add-apt-repository -y ppa:marutter/rrutter")
system("add-apt-repository -y ppa:marutter/c2d4u")
system("apt-get update")
system("apt install -y r-cran-rstan")

### Install packages (all)

In [7]:
install.packages("BiocManager")
install.packages("matrixcalc")
install.packages(c("igraph", "pcalg", "dagitty"), dependencies=TRUE)
install.packages("causaleffect", dependencies=TRUE)
install.packages("devtools", dependencies=TRUE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“dependencies ‘graph’, ‘RBGL’, ‘Rgraphviz’ are not available”
also installing the dependencies ‘zoo’, ‘htmlwidgets’, ‘DEoptimR’, ‘lmtest’, ‘mnormt’, ‘pbivnorm’, ‘numDeriv’, ‘quadprog’, ‘ape’, ‘decor’, ‘igraphdata’, ‘rgl’, ‘vdiffr’, ‘abind’, ‘ggm’, ‘corpcor’, ‘robustbase’, ‘vcd’, ‘bdsmatrix’, ‘sfsmisc’, ‘fastICA’, ‘clue’, ‘RcppArmadillo’, ‘mvtnorm’, ‘huge’, ‘V8’, ‘markdown’, ‘lavaan’


Warning message in install.packages(c("igraph", "pcalg", "dagitty"), dependencies = TRUE):
“installation of package ‘ggm’ had non-zero exit status”
Warning message in install.packages(c("igraph", "pcalg", "dagitty"), dependencies = TRUE):
“installation of package ‘pcalg’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-

In [8]:
BiocManager::install(c("graph", "RBGL", "Rgraphviz"))
devtools::install_github("adele/PAGId", dependencies=TRUE)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.17 (BiocManager 1.30.21), R 4.3.0 (2023-04-21)

Installing package(s) 'BiocVersion', 'graph', 'RBGL', 'Rgraphviz'

also installing the dependency ‘BiocGenerics’


Old packages: 'bit', 'broom', 'bslib', 'curl', 'devtools', 'digest', 'gargle',
  'gert', 'googledrive', 'googlesheets4', 'highr', 'isoband', 'jsonlite',
  'pkgload', 'roxygen2', 'tidyverse', 'usethis', 'whisker', 'boot', 'foreign'




curl     (5.0.0 -> 5.0.1) [CRAN]
jsonlite (1.8.3 -> 1.8.5) [CRAN]
ggm      (NA    -> 2.5  ) [CRAN]
pcalg    (NA    -> 2.7-8) [CRAN]


Skipping 2 packages not available: graph, RBGL

Installing 4 packages: curl, jsonlite, ggm, pcalg

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpgetQTp/remotes5485dc7fe3c/adele-PAGId-1dc096a/DESCRIPTION’ ... OK
* preparing ‘PAGId’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘PAGId_1.0.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



#### Helper functions

In [9]:
library(pcalg)
library(dagitty)
library(MASS)
library(matrixcalc)
library(causaleffect)
library(igraph)
library(PAGId)

####################
# Helper Functions #
####################

# A <- f(Ua) = Ua
# D <- f(Ucd, Ud) = beta_D.Ucd * Ucd + Ud
# B <- f(A, D, Ub) = beta_B.A * A - beta_B.D * D + Ub
# C <- f(Ucd, B, Uc) = beta_C.B * B - beta_C.Ucd * Ucd + Uc
# betaList is a list with entries for:
# beta_D.Ucd, beta_B.A, beta_B.D, beta_C.Ucd, and beta_C.B
# if betaList is NULL, then coefficients are randomly selected
getBDSEM <- function(betaList=NULL) {
  allvars <- c("A", "B", "C", "D", "Ucd")
  p <- length(allvars)
  beta <- matrix(0, p, p)
  colnames(beta) <- rownames(beta) <- allvars
  topolOrd <- c( "Ucd", "A", "D", "B", "C")

  if (is.null(betaList)) {
    betaList <- as.list(sample(c(-1, 1), p, replace = T) * runif(p, 0.3, 0.7))
    names(betaList) <- c("beta_D.Ucd", "beta_B.A", "beta_B.D", "beta_C.Ucd", "beta_C.B")
  }

  beta["Ucd","D"] <- betaList$beta_D.Ucd; # coeff of Ucd in the function for d
  beta["A", "B"] <- betaList$beta_B.A; # coeff of a in the function for b
  beta["D", "B"] <- betaList$beta_B.D; # coeff of d in the function for b
  beta["Ucd", "C"] <- betaList$beta_C.Ucd; # coeff of Ucd in the function for c
  beta["B", "C"] <- betaList$beta_C.B; # coeff of b in the function for c

  # Note that beta is triangular, implying that the SEM is recursive (or acyclic)
  beta <- beta[topolOrd, topolOrd]
  lat <- c("Ucd")

  return(list(beta=beta, lat=lat))
}

# The SEM is constructed as
# Y = beta Y + eps, where
# beta is a matrix of the coefficients for all variables (V and U)
generateDatasetFromSEM <- function(beta, lat, N) {
  p <- ncol(beta)
  ident <- diag(1, p, p)
  colnames(ident) <- rownames(ident) <- colnames(beta)
  # Or, similarly,
  # Y = (I - Beta)^{-1} eps
  # For Gaussian Y and errors eps independent from each other (we have all the Us),
  # we just need to simulate Y from a multivariate distribution
  # of mean zero and cov Sigma=I
  IminusBinv <- ginv(ident - beta)
  Sigma = t(IminusBinv) %*% ident %*% IminusBinv

  valR <- matrixcalc::is.symmetric.matrix(Sigma) &&
    matrixcalc::is.positive.definite(Sigma, tol=1e-8)
  if (!valR) {
    stop("This SEM generates a non-positive definite covariance matrix. Try another SEM.")
  }

  dat <-  MASS::mvrnorm(N, rep(0, p), Sigma, empirical = FALSE)
  dat <- as.data.frame(dat)
  colnames(dat) <- colnames(beta)
  head(dat)

  uvars <- which(colnames(dat) %in% lat)
  dat <- dat[,-uvars]

  return(dat)
}

# Randomly generate a linear SEM following a dagitty DAG, adag,
# and then draw samples from it.
generateDatasetFromDAG <- function(adag, N, ntries=30) {
  done <- FALSE
  tries <- 0
  obs.dat <- NULL
  while (!done && tries <= ntries) {
    done <- tryCatch(
      {
        obs.dat <- dagitty::simulateSEM(adag, b.lower = -0.6, b.upper = 0.6, N=N)
        R <- cor(obs.dat)
        valR <- matrixcalc::is.symmetric.matrix(R) &&
          matrixcalc::is.positive.definite(R, tol=1e-8)
        valR
      }, error=function(cond) {
        message(cond)
        FALSE
      })
    tries <- tries + 1
  }
  return(obs.dat)
}

dagittyOracleCI <- function(x, y, S, suffStat) {
  g <- suffStat$g
  labels <- names(g)
  if (dagitty::dseparated(g, labels[x], labels[y], labels[S])) {
    return(1)
  } else {
    return(0)
  }
}

# receives a dagitty g of type "dag" or "mag" and returns
# the true PAG as an pcalg fci object
getTruePAG <- function(g, verbose = FALSE) {
  indepTest <- dagittyOracleCI
  if (graphType(g) == "dag") {
    g <- dagitty::toMAG(g)
  }
  suffStat <- list(g=g)
  truePag <- pcalg::fci(suffStat,
                        indepTest = indepTest,
                        labels= names(suffStat$g), alpha = 0.9999,
                        verbose = verbose)
  return(truePag)
}

igraph_from_graphNel <- function(graphN, latNodes){
  igraph_dag <- igraph::igraph.from.graphNEL(graphN, weight = FALSE)
  for (n in latNodes) {
    adj_list <- graphN@edgeL[[n]]$edges
    if (length(adj_list) == 2) {
      igraph_dag <- igraph::add_edges(igraph_dag, c(adj_list[1], adj_list[2], adj_list[2], adj_list[1]))
      igraph_dag <- igraph::set.edge.attribute(graph = igraph_dag,
                                               name ="description",
                                               index = c(length(igraph::E(igraph_dag))-1, length(igraph::E(igraph_dag))), value = "U")
    }
  }
  for (n in latNodes){
    igraph_dag <- igraph::delete_vertices(igraph_dag, n)
  }
  return(igraph_dag)
}


# A -> B -> C; B <- D <- Ucd -> C
getBDGraph <- function() {
  allvars <- c("A", "B", "C", "D", "Ucd")
  p <- length(allvars)
  amat <- matrix(0, p, p)
  colnames(amat) <- rownames(amat) <- allvars
  amat["A","B"] <- 0; amat["B","A"] <- 1; # a -> b
  amat["B","C"] <- 0; amat["C","B"] <- 1; # b -> c
  amat["D","B"] <- 0; amat["B","D"] <- 1; # d -> b
  amat["Ucd","C"] <- 0; amat["C","Ucd"] <- 1; # Ucd -> c
  amat["Ucd","D"] <- 0; amat["D","Ucd"] <- 1; # Ucd -> c

  lat <- c("Ucd")
  adag <- pcalg::pcalg2dagitty(amat, colnames(amat), type="dag")
  dagitty::latents(adag) <- lat

  return(list(adag=adag, amat=amat, lat=lat))
}

# DAG in Fig. 2b) at https://causalai.net/r42.pdf
getDAG2bR42 <- function(ret_dagg = TRUE) {
  allvars <- c("x1", "x2", "y1", "y2", "y3", "y4", "y5", "ux1x2", "uy2y3", "uy4y5", "ux1y3", "ux2y1")
  p <- length(allvars)
  amat <- matrix(0, p, p)
  colnames(amat) <- rownames(amat) <- allvars
  amat["x2", "y2"] <- 0; amat["y2", "x2"] <- 1; # x2 -> y2
  amat["x1", "y1"] <- 0; amat["y1", "x1"] <- 1; # x1 -> y1
  amat["y4", "y3"] <- 0; amat["y3", "y4"] <- 1; # y4 -> y3
  amat["y5", "y1"] <- 0; amat["y1", "y5"] <- 1; # y5 -> y1
  amat["y5", "y4"] <- 0; amat["y4", "y5"] <- 1; # y5 -> y4
  amat["ux1x2", "x1"] <- 0; amat["x1", "ux1x2"] <- 1; # ux1x2 -> x1
  amat["ux1x2", "x2"] <- 0; amat["x2", "ux1x2"] <- 1; # ux1x2 -> x2
  amat["uy2y3", "y2"] <- 0; amat["y2", "uy2y3"] <- 1; # uy2y3 -> y2
  amat["uy2y3", "y3"] <- 0; amat["y3", "uy2y3"] <- 1; # uy2y3 -> y3
  amat["uy4y5", "y4"] <- 0; amat["y4", "uy4y5"] <- 1; # uy4y5 -> y4
  amat["uy4y5", "y5"] <- 0; amat["y5", "uy4y5"] <- 1; # uy4y5 -> y5
  amat["ux1y3", "x1"] <- 0; amat["x1", "ux1y3"] <- 1; # ux1y3 -> x1
  amat["ux1y3", "y3"] <- 0; amat["y3", "ux1y3"] <- 1; # ux1y3 -> y3
  amat["ux2y1", "x2"] <- 0; amat["x2", "ux2y1"] <- 1; # ux2y1 -> x2
  amat["ux2y1", "y1"] <- 0; amat["y1", "ux2y1"] <- 1; # ux2y1 -> y1
  # plot(as(t(amat), "graphNEL"))

  lat <- c("ux1x2", "uy2y3", "uy4y5", "ux1y3", "ux2y1")

  adag <- pcalg::pcalg2dagitty(amat, colnames(amat),type="dag")

  dagitty::latents(adag) <- lat
  return(list(adag=adag, amat=amat, lat=lat))
}

# returns an pcalg amat (adjacency matrix) of type amat.pag (same type for MAGs),
# where:
# 0: No edge
# 1: Circle
# 2: Arrowhead
# 3: Tail
dagitty2amat <- function(adagg, type="mag") {
  edg <- dagitty:::edges(adagg)
  node_names <- dagitty:::names.dagitty(adagg)
  ans_mat <- matrix(
    data = 0, nrow = length(node_names),
    ncol = length(node_names),
    dimnames = list(node_names, node_names)
  )

  diredg <- subset(edg, e == "->")

  ans_mat[as.matrix(diredg[c("w", "v")])] <- 3
  ans_mat[as.matrix(diredg[c("v", "w")])] <- 2

  bidiredg <-  subset(edg, e == "<->")
  ans_mat[as.matrix(bidiredg[c("w", "v")])] <- 2
  ans_mat[as.matrix(bidiredg[c("v", "w")])] <- 2

  return(ans_mat)
}

# type can be "pag" or "dag"
getMAG <- function(amat, type="pag") {
  if (type == "pag") {
    amat.mag <- pcalg::pag2magAM(amat, 1)
    #plotAG(amat.mag)
    magg <- pcalg::pcalg2dagitty(amat.mag, colnames(amat.mag), type="mag")
    #plot(magg)
  } else {
    adagg <- pcalg::pcalg2dagitty(amat, colnames(amat), type="dag")
    magg <- dagitty::toMAG(adagg)
    amat.mag <- dagitty2amat(magg, type="mag")
  }
  return(list(amat.mag = amat.mag, magg=magg))
}



Attaching package: ‘dagitty’


The following object is masked from ‘package:pcalg’:

    randomDAG



Attaching package: ‘causaleffect’


The following object is masked from ‘package:utils’:

    recover



Attaching package: ‘igraph’


The following object is masked from ‘package:matrixcalc’:

    %s%


The following object is masked from ‘package:dagitty’:

    edges


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union




# Running the example

#### 1. Constructing the Structural Causal Model and corresponding Causal Diagram

In [1]:
#########################################
# Constructing the true Causal Diagram  #
#########################################

# Getting the true causal diagram
trueDAG.out <- getDAG2bR42()
trueDAG.dag <- trueDAG.out$adag # a dagitty object

# Note: this plots the DAG including the U's variables
plot(trueDAG.dag)

# Getting the ADMG corresponding to the true DAG
trueDAG.amat <- trueDAG.out$amat # the adjacency matrix following the pcalg notation
trueDAG.gNEL <- as(t(trueDAG.out$amat), "graphNEL")
trueADMG.ig <- igraph_from_graphNel(trueDAG.gNEL, latents(trueDAG.dag))

# Note: this plots the causal diagram with bidirected edges indicating unmeasured confounders
plot(trueADMG.ig)


ERROR: ignored

#### 2. Checkings for ADMG/MAG

In [ ]:
###################################################
# Checking the conditional independence relations #
# implied by the true Causal Diagram              #
###################################################

# Getting the minimal conditional independence relationships over the observed variables,
# implied by the true ADMG -- obtained by minimal d-separators
trueImpliedCI <- dagitty::impliedConditionalIndependencies(trueDAG.dag, type = "missing.edge")
print(trueImpliedCI)


#### 3. Causal Effect Identification from the ADMG

##### 3.A. Identification through Adjustment

In [ ]:
################################################################
# Checking if P(y1,y2,y3,y4,y5|do(x1,x2)) is identifiable      #
# from the ADMG/MAG using the generalized adjustment criterion #
################################################################

x <- 1:2
y <- 3:7

mag <- dagitty::toMAG(trueDAG.dag)
amat.mag <- dagitty2amat(mag, type="mag")

adj <- adjustment(amat = amat.mag,
                  amat.type = "mag", x = x, y = y,
                  set.type = "all")
print(adj) # There is no valid adjustment set for X={X1,X2} and Y={Y1,Y2,Y3,Y4,Y5}


##### 3.B. Identification through Id Algorithm

In [ ]:

########################################################
# Identifying causal effect P(y |do(x)) from the ADMG  #
########################################################

x <- c("x1", "x2")
y <- c("y1", "y2", "y3", "y4", "y5")
z <- NULL


exprDAG <- causaleffect::causal.effect(y=y, x=x, z=z,
                                       G = trueADMG.ig,
                                       expr = TRUE,
                                       simp = TRUE)
print(paste0("ID from DAG: ", exprDAG))

# to see all steps applied by the ID algorithm,
# use expr = FALSE and steps = TRUE.
retDAG <- causaleffect::causal.effect(y=y, x=x, z=z,
                                      G = trueADMG.ig,
                                      expr = FALSE,
                                      simp = TRUE,
                                      steps = TRUE)
#print(retDAG)


#### 4. Simulating data from the true DAG

In [ ]:
#####################################
# Simulating data from the true DAG #
#####################################

N = 10000 # try different values of N
#seed = ceiling(runif(1, 0, 10000))
seed = 3632
set.seed(seed)

# The following function generates a random linear Gaussian SCM
# compatible with the true DAG and then samples from it:
dat <- generateDatasetFromDAG(adag = trueDAG.dag, N=N)
head(dat)


#### 5. Checking expected dependencies and independencies in data

In [ ]:

#########################################################
# Testing some expected dependencies and independencies #
#########################################################

# For Gaussian data, partial correlation tests (Fisher's Z test)
# can be used to assess conditional independencies
# For other data types, appropriate conditional independence tests should be used.
indepTest <- pcalg::gaussCItest
alpha <- 0.05
suffStat <- list(C = cor(dat), n = N)

X1ind <- 1
X2ind <- 2
Y1ind <- 3
Y2ind <- 4
Y3ind <- 5
Y4ind <- 6
Y5ind <- 7

# some dependencies:
p_x1y4.y3 <- indepTest(X1ind, Y4ind, Y3ind, suffStat)
p_x1y5.y1 <- indepTest(X1ind, Y5ind, Y1ind, suffStat)
p_x2y3.y2 <- indepTest(X2ind, Y3ind, Y2ind, suffStat)
p_y3y1.y5 <- indepTest(Y3ind, Y1ind, Y5ind, suffStat)
p_y3y5 <- indepTest(Y3ind, Y5ind, c(), suffStat)

dep_p <- c(p_x1y4.y3, p_x1y5.y1, p_x2y3.y2, p_y3y1.y5, p_y3y5)
print(all(dep_p < alpha))


# minimal indepedencies
p_x1y2.x2 <- indepTest(X1ind, Y2ind, X2ind, suffStat)
p_x1y4 <- indepTest(X1ind, Y4ind, c(), suffStat)
p_x1y5 <- indepTest(X1ind, Y5ind, c(), suffStat)
p_x2y3 <- indepTest(X2ind, Y3ind, c(), suffStat)
p_x2y4 <- indepTest(X2ind, Y4ind, c(), suffStat)
p_x2y5 <- indepTest(X2ind, Y5ind, c(), suffStat)

p_y1y2.x2 <- indepTest(Y1ind, Y2ind, X2ind, suffStat)
p_y1y3.x1y4 <- indepTest(Y1ind, Y3ind, c(X1ind, Y4ind), suffStat)
p_y1y3.x1y5 <- indepTest(Y1ind, Y3ind, c(X1ind, Y5ind), suffStat)
p_y1y4.y5 <- indepTest(Y1ind, Y4ind, Y5ind, suffStat)

p_y2y4 <- indepTest(Y2ind, Y4ind, c(), suffStat)
p_y2y5 <- indepTest(Y2ind, Y5ind, c(), suffStat)
p_y3y5.y4 <- indepTest(Y3ind, Y5ind, Y4ind, suffStat)

indep_p <- c(p_x1y2.x2, p_x1y4, p_x1y5, p_x2y3, p_x2y4, p_x2y5, p_y1y2.x2,
      p_y1y3.x1y4, p_y1y3.x1y5, p_y1y4.y5, p_y2y4, p_y2y5, p_y3y5.y4)
print(all(indep_p >= alpha))

#### 6. Causal Discovery: Estimating a PAG using **FCI**

In [ ]:
###############################
# Estimating a PAG using FCI  #
###############################

# Get the PAG using the true ADMG as an oracle of cond. indep. relationships
truePAG <- getTruePAG(trueDAG.dag, verbose = TRUE)
plot(truePAG)


# For Gaussian data, partial correlation tests (Fisher's Z test)
# can be used to assess conditional independencies
# For other data types, appropriate conditional independence tests should be used.
indepTest <- pcalg::gaussCItest
alpha <- 0.05
suffStat <- list(C = cor(dat), n = N)
estPAG <- pcalg::fci(suffStat,
                     indepTest = indepTest,
                     labels= colnames(dat), alpha = alpha,
                     verbose = TRUE)
plot(estPAG)

# True and estimated PAGs are identical
print(all(truePAG@amat - estPAG@amat == 0))

#### 7. Checkings for the PAG

In [ ]:
###################################################
# Checking the conditional independence relations #
# implied by the estimated PAG                    #
###################################################

# Getting the minimal conditional independence relationships
# implied by the estimated PAG
# This can be obtained by computing the minimal d-separators in
# any MAG that is member of the PAG
amat.mag <- pcalg::pag2magAM(estPAG@amat, 1)
plotAG(amat.mag)
amag <- pcalg::pcalg2dagitty(amat.mag, colnames(amat.mag), type="mag")

estImpliedCI <- dagitty::impliedConditionalIndependencies(amag)
print(estImpliedCI)


#### 8. Causal Effect Identification from the PAG

##### 8.A. Identification through Adjustment 

In [ ]:

###########################################################
# Checking if P(y1,y2,y3,y4,y5|do(x1,x2)) is identifiable #
# from the PAG using generalized adjustment criterion.    #
###########################################################

#P_{x1,x2}(y1,y2,y3,y4,y5)
Xinds <- c(1,2)
Yinds <- 3:7

x <- Xinds
y <- Yinds

# adjacency matrix of the estimated PAG following the pcalg notation
estPAG.amat <- estPAG@amat

adj <- adjustment(amat = estPAG.amat, amat.type = "pag", x = x, y = y, set.type = "all")
print(adj) # There is no valid adjustment set for X={A, F} and Y={Y}


# Testing if the particular set {} is valid for adjustment:
z <- c()
gac_z <- gac(estPAG.amat, x, y, z, type = "pag")
print(gac_z$gac) # the set is not valid for adjustment

# the third condition of the gac is violated, i.e.,
# not all proper definite status non-causal paths from x to y are blocked by z
print(gac_z$res)




##### 8.B Identification through PAG-Id Algorithm 

In [ ]:
########################################################
# Identifying P(y1,y2,y3,y4,y5|do(x1,x2)) from the PAG #
# using the CIDP algorithm                             #
########################################################

x <- c("x1", "x2")
y <- c("y1", "y2", "y3", "y4", "y5")
z <- NULL


# adjacency matrix of the estimated PAG following the pcalg notation
estPAG.amat <- estPAG@amat

retPAG <- CIDP(estPAG.amat, x, y, z)
print(retPAG$id)
print(paste0("ID from PAG: ", retPAG$Qexpr[[retPAG$query]]))

# This shows the steps taken by the CIDP algorithm
# by substitution and simplification, we will get the same
# identification formula from the ADMG
print(retPAG$Qexpr)